In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Don't forget to add the current directory to pythonpath before loading the package
# $ export PYTHONPATH=$PYTHONPATH:$(pwd)
# & jupyter notebook
from aphp import load

# Qualité des données

- pas de valeurs manquantes
- on choisit de reformatter la colonne `pcr` en booléen en prenant en compte
  les différents textes (exemple, `'Positive' == 'P' == True`)
- tous les IDs dans cette table sont présents dans la table des patients. Malheureusement on perd 1.5 % des résultats
  de tests PCR lorsque l'on supprime les patients avec un ID erronné (on ne sait pas à qui attribuer un ID parmis 2
  patients)
  
 

In [3]:
df = load.load_raw_tests()

In [4]:
len(df)

8800

In [5]:
df.dtypes

patient_id     int64
pcr           object
dtype: object

In [6]:
df.isnull().mean() * 100

patient_id    0.0
pcr           0.0
dtype: float64

In [7]:
df['pcr'].value_counts()

N           3482
Negative    3134
Positive    1283
P            901
Name: pcr, dtype: int64

In [8]:
df['pcr_positive'] = df['pcr'].str.capitalize().str.slice(0, 1) == "P"
df['pcr_positive'].value_counts()

False    6616
True     2184
Name: pcr_positive, dtype: int64

In [9]:
1283 + 901

2184

In [10]:
df['patient_id'].duplicated().sum()

35

In [11]:
df.loc[df['patient_id'].duplicated(keep=False)].sort_values(by='patient_id')

,patient_id,pcr,pcr_positive
169,159500,N,False
2934,159500,N,False
8469,169519,Positive,True
1842,169519,Negative,False
5618,212217,N,False
...,...,...,...
5542,887646,Negative,False
1025,975512,Negative,False
2331,975512,N,False
7477,990695,N,False


In [12]:
df = df.groupby('patient_id')['pcr_positive'].any().astype(bool).reset_index()

In [13]:
patient_raw = load.load_raw_patients()
patient_clean = load.load_clean_patients()

In [14]:
df['patient_id'].isin(patient_raw['patient_id'].unique()).mean() * 100

100.0

In [15]:
df['patient_id'].isin(patient_clean['patient_id'].unique()).mean() * 100

98.48260125499144

In [16]:
df['pcr_positive'].value_counts()

False    6583
True     2182
Name: pcr_positive, dtype: int64